In [99]:
import json
import requests
import numpy as np
import pandas as pd 
import regex as re
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import string
from pandas.api.types import CategoricalDtype

In [35]:

site_url = 'https://stlouis.myfeelstate.com/graphql?operationName=FilteredProducts&variables={"includeEnterpriseSpecials":false,"includeCannabinoids":true,"productsFilter":{"dispensaryId":"600f5a29d692585e7ffd25e4","pricingType":"med","strainTypes":[],"subcategories":[],"Status":"Active","types":[],"useCache":false,"sortDirection":1,"sortBy":null,"isDefaultSort":true,"bypassOnlineThresholds":false,"isKioskMenu":false,"removeProductsBelowOptionThresholds":true},"page":0,"perPage":500}&extensions={"persistedQuery":{"version":1,"sha256Hash":"aed681974c9093e0cc3099ef18a8d37e17e4916a9f7743dd372e0be6b13c8f10"}}'
r= requests.get(site_url)
data = r.json()

In [36]:
dataframe = {'Type':[],'Brand':[],'Name':[],'Weight':[], 'THC_content':[], 'CBD_content':[],'Price':[]}
product_data= data['data']['filteredProducts']['products']

In [374]:

site_url = 'https://stlouis.myfeelstate.com/graphql?operationName=FilteredProducts&variables={"includeEnterpriseSpecials":false,"includeCannabinoids":true,"productsFilter":{"dispensaryId":"600f5a29d692585e7ffd25e4","pricingType":"med","strainTypes":[],"subcategories":[],"Status":"Active","types":[],"useCache":false,"sortDirection":1,"sortBy":null,"isDefaultSort":true,"bypassOnlineThresholds":false,"isKioskMenu":false,"removeProductsBelowOptionThresholds":true},"page":0,"perPage":500}&extensions={"persistedQuery":{"version":1,"sha256Hash":"aed681974c9093e0cc3099ef18a8d37e17e4916a9f7743dd372e0be6b13c8f10"}}'
r= requests.get(site_url)
data = r.json()

dataframe = {'Type':[],'Brand':[],'Name':[],'Weight':[], 'THC_content':[], 'CBD_content':[],'Price':[]}
product_data= data['data']['filteredProducts']['products']

def retrieve_name_weight(json):
    global product
    global weight
    product= item['Name']
    product= re.sub('[-|]', '', product)
    weight = re.search('\d.\d+g$|\d+g$|\d*mg$|^ \d+.*ck$|\d+ml$', product)
    weight_measurement= 'mg' if item['measurements']['netWeight']['unit'] == 'MILLIGRAMS' else ''
    
    if brand != None:
        product = re.sub(brand, '', product)
    else:
        pass
    
    if weight!= None:
        weight = weight.group()
        product = re.sub(weight, '',product)
        dataframe['Name'].append(product)
        dataframe['Weight'].append(weight)
    elif (item['measurements']['netWeight']['values'] != []) and (min(item['measurements']['netWeight']['values'])) > 0:
        weight= ''.join(str(item['measurements']['netWeight']['values']).strip('[]')+ weight_measurement)
        dataframe['Weight'].append(weight)
        dataframe['Name'].append(product.strip())
    else:
        dataframe['Weight'].append(' ')
        dataframe['Name'].append(product.strip())   

def CBD_THC_content(json):
    global CBD_content
    global THC_content
    
    if (item['CBDContent'] != None) and (item['CBDContent']['range'] != None and max(item['CBDContent']['range']) > 0 ):
        CBD_content =  str(max(item['CBDContent']['range'])).strip('[]')+' '+ item['CBDContent']['unit']
        dataframe['CBD_content'].append(CBD_content.title())
    else:
        dataframe['CBD_content'].append(' ')
    
    if (item['THCContent'] != None) and (item['THCContent']['range'] != None and max(item['THCContent']['range']) > 0 ):
        THC_content = str(max(item['THCContent']['range'])).strip('[]')+' '+ item['THCContent']['unit']
        dataframe['THC_content'].append(THC_content.title())
    else:
        dataframe['THC_content'].append(' ')

def retrieve_price(json):
    global price
    
    if item['special'] == True:
        price = '{:.2f}'.format(max(item['recSpecialPrices']))
    else:
        price = '{:.2f}'.format(min(item['Prices']))

def get_range(clean_data):
    range_end= [(i,word) for i,word in zip(string.ascii_uppercase, clean_data.columns.values)][-1][0]
    global table_range
    table_range = f'$A$2:${range_end}${str(len(clean_data.index)+1)}'

def custom_sort(df):
    # How the Categories are to be displayed 
    sort_types = CategoricalDtype(['Vaporizers','Edible','Concentrate','Topicals','Tincture','Accessories','Apparel'], ordered =True)
    
    # Seperates house brand flowers from non house brands(n_h_flower) sorts them and concats them back together
    house_flower = df.loc[(df['Type'] == 'Flower')&(df['Brand'].isin(['Robust','Packs Cannabis']))].sort_values(by=['Brand','Name'], ascending=[False, True])
    n_h_flower = df.loc[(df['Type'] == 'Flower')&(~df['Brand'].isin(['Robust','Packs Cannabis']))].sort_values(by=['Brand','Name'])
    flower = pd.concat([house_flower, n_h_flower])

    # Seperates house brand prerolls from non house prerolls(n_h_prerolls) sorts them and concats them back together
    house_prerolls = df.loc[(df['Type'] == 'Pre-Rolls')&(df['Brand']=='Heya')].sort_values(by=['Brand','Name'])
    n_h_prerolls = df.loc[(df['Type'] == 'Pre-Rolls')& (df['Brand'] != 'Heya')].sort_values(by=['Brand','Name'])
    prerolls = pd.concat([house_prerolls, n_h_prerolls])

    drop_list = list(flower.index) + list(prerolls.index)
    non_house_brands = df.drop(labels=drop_list)
    non_house_brands['Type'] = non_house_brands['Type'].astype(sort_types)
    
    non_house_brands= non_house_brands.sort_values(by=['Type','Brand','Name'])
    
    global clean_data
    clean_data= pd.concat([flower,prerolls,non_house_brands])

for item in product_data:
    # Removing brand from name 
    try:
        brand = item['POSMetaData']['canonicalBrandName']
        dataframe['Brand'].append(brand)
    except:
        pass
    
    retrieve_name_weight(item)
    retrieve_price(item)
    
    if item['type'] != 'Accessories' or 'Apparel':
        CBD_THC_content(item)
    else:
        pass
    
    dataframe['Type'].append(item['type'])
    dataframe['Price'].append(price)
    
df =pd.DataFrame.from_dict(data=dataframe)
custom_sort(df)
get_range(clean_data)


client = gspread.service_account(r'C:\Users\bboul\Desktop\feel_state_key.json')

workbook= client.open('Feel State-Menu')
worksheet = workbook.get_worksheet(0)

worksheet.update(table_range, clean_data.to_json())




ValueError: DataFrame index must be unique for orient='columns'.

In [391]:
clean_data.head(40)

,Brand,Name,Weight,THC_content,CBD_content,Price
Type,,,,,,
Flower,Robust,Apple Cinnamon Crumble,3.5g,3.66 Percentage,9.39 Percentage,16.00
Flower,Robust,Apple Fritter,3.5g,13.74 Percentage,0.06 Percentage,20.00
Flower,Robust,Grape LSD,3.5g,9.56 Percentage,0.3 Percentage,16.00
Flower,Robust,Yuhbary #26,3.5g,13.21 Percentage,0.08 Percentage,20.00
Flower,Packs Cannabis,Pink Champagne,7g,11.66 Percentage,0.31 Percentage,28.00
Flower,Packs Cannabis,Purple Gorilla,7g,15.62 Percentage,,32.00
Flower,Packs Cannabis,Royal Cherry Kush,7g,11.54 Percentage,,28.00
Flower,Beach,Citrus Palisade,7g,17.48 Percentage,0.39 Percentage,40.00
Flower,Beach,Clementine,7g,20.13 Percentage,0.04 Percentage,40.00


In [340]:
df.head()

,Type,Brand,Name,Weight,THC_content,CBD_content,Price
Type,,,,,,,
Accessories,Accessories,LuvBuds,Mushroom Carb Cap Assorted Colors,,,,4.00
Accessories,Accessories,None,Vibes King Size Slim Cones (Blue),,,,3.00
Accessories,Accessories,RAW,Raw King Size Slim,,,,1.60
Accessories,Accessories,None,Yocan Revolve,,,,28.00
Accessories,Accessories,Handwraps Papers,Handwraps 1/4,,,,2.00


In [390]:
dataframe = {'Type':[],'Brand':[],'Name':[],'Weight':[], 'THC_content':[], 'CBD_content':[],'Price':[]}
for item in product_data:
    # Removing brand from name 
    try:
        brand = item['POSMetaData']['canonicalBrandName']
        dataframe['Brand'].append(brand)
    except:
        pass
    
    retrieve_name_weight(item)
    retrieve_price(item)
    
    if item['type'] != 'Accessories' or 'Apparel':
        CBD_THC_content(item)
    else:
        pass
    
    dataframe['Type'].append(item['type'])
    dataframe['Price'].append(price)
    
df =pd.DataFrame.from_dict(data=dataframe)
custom_sort(df)
get_range(clean_data)
clean_data.set_index('Type', inplace=True)
clean_data.fillna('')

client = gspread.service_account(r'C:\Users\bboul\Desktop\feel_state_key.json')

workbook= client.open('Feel State-Menu')
worksheet = workbook.get_worksheet(0)

worksheet.update(table_range, clean_data.values.tolist())

{'spreadsheetId': '1fqHezMY9PNw2dj3zvmLiCUoj6OWvugko5sILbAlCmfI',
 'updatedRange': "'Feel-State'!A2:F345",
 'updatedRows': 344,
 'updatedColumns': 6,
 'updatedCells': 2014}

In [ ]:
dataframe = {'Type':[],'Brand':[],'Name':[],'Weight':[], 'THC_content':[], 'CBD_content':[],'Price':[]}

In [376]:
dataframe = {'Type':[],'Brand':[],'Name':[],'Weight':[], 'THC_content':[], 'CBD_content':[],'Price':[]}
for i in dataframe:
    print(dataframe[i])

[]
[]
[]
[]
[]
[]
[]


In [323]:
df =pd.DataFrame.from_dict(data=dataframe)
get_range(df)
custom_sort(df)

client = gspread.service_account(r'C:\Users\bboul\Desktop\feel_state_key.json')

workbook= client.open('Feel State-Menu')
worksheet = workbook.get_worksheet(0)

worksheet.update(table_range, clean_data.values.tolist())


InvalidJSONError: Out of range float values are not JSON compliant

In [309]:
df =pd.DataFrame.from_dict(data=dataframe)
get_range(df)

In [316]:
df.head()

,Type,Brand,Name,Weight,THC_content,CBD_content,Price
0,Edible,Care by Design,4:1 CBD Soft Gels 10Pack,25mg,29.24 Milligrams,11.7 Milligrams,30.40
1,Accessories,Jefe,Hempen Roach Clip,,,,24.00
2,Concentrate,Melee On The Mississippi,Melee On the Mississippi,,,,100.00
3,Flower,Mama J's,Sunset MAC,3.5g,23.44 Percentage,,32.00
4,Edible,Wana Gummies,1:1 Strawberry Lemonade Sour Gummies 10pack,100mg,100 Milligrams,100 Milligrams,25.60


In [317]:

def custom_sort(df):
    # How the Categories are to be displayed 
    sort_types = CategoricalDtype(['Vaporizers','Edible','Concentrate','Topicals','Tincture','Accessories','Apparel'], ordered =True)
    
    # Seperates house brand flowers from non house brands(n_h_flower) sorts them and concats them back together
    house_flower = df.loc[(df['Type'] == 'Flower')&(df['Brand'].isin(['Robust','Packs Cannabis']))].sort_values(by=['Brand','Name'], ascending=[False, True])
    n_h_flower = df.loc[(df['Type'] == 'Flower')&(~df['Brand'].isin(['Robust','Packs Cannabis']))].sort_values(by=['Brand','Name'])
    flower = pd.concat([house_flower, n_h_flower])

    # Seperates house brand prerolls from non house prerolls(n_h_prerolls) sorts them and concats them back together
    house_prerolls = df.loc[(df['Type'] == 'Pre-Rolls')&(df['Brand']=='Heya')].sort_values(by=['Brand','Name'])
    n_h_prerolls = df.loc[(df['Type'] == 'Pre-Rolls')& (df['Brand'] != 'Heya')].sort_values(by=['Brand','Name'])
    prerolls = pd.concat([house_prerolls, n_h_prerolls])

    drop_list = list(flower.index) + list(prerolls.index)
    non_house_brands = df.drop(labels=drop_list)
    non_house_brands['Type'] = non_house_brands['Type'].astype(sort_types)
    
    non_house_brands= non_house_brands.sort_values(by=['Type','Brand','Name'])
    
    global clean_data
    clean_data= pd.concat([flower,prerolls,non_house_brands])
custom_sort(df)

## Data Scrapping Block 

In [ ]:
def retrieve_name_weight(json):
    global product
    global weight
    product= item['Name']
    product= re.sub('[-|]', '', product)
    weight = re.search('\d.\d+g$|\d+g$|\d*mg$|^ \d+.*ck$|\d+ml$', product)
    weight_measurement= 'mg' if item['measurements']['netWeight']['unit'] == 'MILLIGRAMS' else ''
    
    if brand != None:
        product = re.sub(brand, '', product)
    else:
        pass
    
    if weight!= None:
        weight = weight.group()
        product = re.sub(weight, '',product)
        dataframe['Name'].append(product)
        dataframe['Weight'].append(weight)
    elif (item['measurements']['netWeight']['values'] != []) and (min(item['measurements']['netWeight']['values'])) > 0:
        weight= ''.join(str(item['measurements']['netWeight']['values']).strip('[]')+ weight_measurement)
        dataframe['Weight'].append(weight)
        dataframe['Name'].append(product.strip())
    else:
        dataframe['Weight'].append(' ')
        dataframe['Name'].append(product.strip())   

def CBD_THC_content(json):
    global CBD_content
    global THC_content
    
    if (item['CBDContent'] != None) and (item['CBDContent']['range'] != None and max(item['CBDContent']['range']) > 0 ):
        CBD_content =  str(max(item['CBDContent']['range'])).strip('[]')+' '+ item['CBDContent']['unit']
        dataframe['CBD_content'].append(CBD_content.title())
    else:
        dataframe['CBD_content'].append(' ')
    
    if (item['THCContent'] != None) and (item['THCContent']['range'] != None and max(item['THCContent']['range']) > 0 ):
        THC_content = str(max(item['THCContent']['range'])).strip('[]')+' '+ item['THCContent']['unit']
        dataframe['THC_content'].append(THC_content.title())
    else:
        dataframe['THC_content'].append(' ')

def retrieve_price(json):
    global price
    
    if item['special'] == True:
        price = '{:.2f}'.format(max(item['recSpecialPrices']))
    else:
        price = '{:.2f}'.format(min(item['Prices']))

def get_range(df):
    range_end= [(i,word) for i,word in zip(string.ascii_uppercase, df.columns.values)][-1][0]
    global table_range
    table_range = f'$A$2:${range_end}${str(len(df.index))}'

for item in product_data:
    # Removing brand from name 
    try:
        brand = item['POSMetaData']['canonicalBrandName']
        dataframe['Brand'].append(brand)
    except:
        pass
    
    retrieve_name_weight(item)
    retrieve_price(item)
    
    if item['type'] != 'Accessories' or 'Apparel':
        CBD_THC_content(item)
    else:
        pass
    
    dataframe['Type'].append(item['type'])
    dataframe['Price'].append(price)
    
df =pd.DataFrame.from_dict(data=dataframe)
df.set_index(df['Type'], inplace = True)
df = df.sort_index()
get_range(df):

client = gspread.service_account(r'C:\Users\bboul\Desktop\feel_state_key.json')

workbook= client.open('Feel State-Menu')
worksheet = workbook.get_worksheet(0)

worksheet.update(table_range, df.values.tolist())

## Google Upload Block

In [ ]:
values=df.values

In [ ]:
df.dtypes

Type           object
Brand          object
Name           object
Weight         object
THC_content    object
CBD_content    object
Price          object
dtype: object

In [ ]:
for i in df.columns:
    print(i)
    df[i]= df[i].astype(str)

Type
Brand
Name
Weight
THC_content
CBD_content
Price


In [ ]:
print(table_range)

$A$2:$G$349 {'Type': ['Edible', 'Accessories', 'Concentrate', 'Flower', 'Edible', 'Tincture', 'Edible', 'Accessories', 'Accessories', 'Accessories', 'Accessories', 'Accessories', 'Tincture', 'Accessories', 'Edible', 'Accessories', 'Accessories', 'Concentrate', 'Concentrate', 'Flower', 'Pre-Rolls', 'Vaporizers', 'Concentrate', 'Flower', 'Flower', 'Pre-Rolls', 'Pre-Rolls', 'Pre-Rolls', 'Vaporizers', 'Vaporizers', 'Edible', 'Edible', 'Concentrate', 'Vaporizers', 'Vaporizers', 'Concentrate', 'Vaporizers', 'Pre-Rolls', 'Vaporizers', 'Accessories', 'Edible', 'Concentrate', 'Edible', 'Edible', 'Vaporizers', 'Vaporizers', 'Flower', 'Edible', 'Edible', 'Edible', 'Concentrate', 'Edible', 'Concentrate', 'Vaporizers', 'Edible', 'Vaporizers', 'Topicals', 'Flower', 'Flower', 'Edible', 'Concentrate', 'Flower', 'Edible', 'Edible', 'Flower', 'Flower', 'Flower', 'Vaporizers', 'Edible', 'Flower', 'Concentrate', 'Vaporizers', 'Pre-Rolls', 'Pre-Rolls', 'Vaporizers', 'Concentrate', 'Flower', 'Pre-Rolls', 'V

In [ ]:
import gspread

client = gspread.service_account(r'C:\Users\bboul\Desktop\feel_state_key.json')

workbook= client.open('Feel State-Menu')
worksheet = workbook.get_worksheet(0)

worksheet.update(table_range, values.tolist())
     

{'spreadsheetId': '1fqHezMY9PNw2dj3zvmLiCUoj6OWvugko5sILbAlCmfI',
 'updatedRange': "'Feel-State'!A2:G346",
 'updatedRows': 345,
 'updatedColumns': 7,
 'updatedCells': 2415}

In [ ]:
df.head()

,Type,Brand,Name,Weight,THC_content,CBD_content,Price
Type,,,,,,,
Accessories,Accessories,Feel State,Lighter,,,,2.40
Accessories,Accessories,LuvBuds,Scortch Torch with Adjustable Flame 6.5in,,,,16.00
Accessories,Accessories,Eyce,Shorty Assorted Colors,,,,8.00
Accessories,Accessories,LuvBuds,Silicone Pipe Replacement Glass,,,,4.00
Accessories,Accessories,None,"Small, Black Accent Beaker Base Water Pipe",,,,100.00


In [ ]:
df.to_csv(r'C:\Users\bboul\OneDrive\Desktop\Feel-State',index=False)

In [ ]:
data

{'data': {'filteredProducts': {'products': [{'_id': '6221507fdbe7ff000c804c77',
     'id': '6221507fdbe7ff000c804c77',
     'AdditionalOptions': [],
     'brandId': '600b849e-87dd-4f84-914a-2c848cb99be9',
     'brandName': 'Care By Design',
     'CBD': None,
     'CBDContent': {'unit': 'MILLIGRAMS',
      'range': [0, 11.7],
      '__typename': 'ProductPotency'},
     'comingSoon': None,
     'createdAt': '1646350463466',
     'DispensaryID': '600f5a29d692585e7ffd25e4',
     'enterpriseProductId': '631a0b1bbf640b000c3536ca',
     'Image': 'https://s3-us-west-2.amazonaws.com/dutchie-images/81aa2f920643670fc2524f5ffba7b0c2',
     'images': [{'url': 'https://s3-us-west-2.amazonaws.com/dutchie-images/81aa2f920643670fc2524f5ffba7b0c2',
       'description': '',
       'active': True,
       '__typename': 'Products_Images'},
      {'url': 'https://s3-us-west-2.amazonaws.com/dutchie-images/22e6de4a4d6229bb47b6279c0af3720b',
       'description': '',
       'active': True,
       '__typename':